In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
df= pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

There are 19 columns

In [4]:
# deriving trip duration in minutes
df['duration']= df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']
df['duration']=df['duration'].apply(lambda x: x.total_seconds()/60)

In [5]:
# standard deviation of trip duration
df['duration'].std()

42.594351241920904

In [6]:
#Filter dataset to contain only trips with duration between 1 and 60 minutes
new_df=df[(df['duration']>= 1) & (df['duration']<= 60)]

# fraction of trips within selected duration
len(new_df)/len(df)

0.9812202822125979

In [7]:
variables=new_df[['PULocationID' ,'DOLocationID']].astype(str)
train_dict=variables.to_dict(orient='records')

In [8]:
dv=DictVectorizer()
x_train=dv.fit_transform(train_dict)
y_train=new_df['duration']

In [9]:
x_train.shape

(3009173, 515)

In [10]:
model= LinearRegression()
model.fit(x_train, y_train)
y_pred=model.predict(x_train)

In [ ]:
with open('model/lin_reg.bin', wb) as f_out:
    pickle.dump((model,dv), f_out)

In [12]:
import math
#root mean squared error= root of mean squared error
rmse=math.sqrt(mean_squared_error(y_train, y_pred))
rmse

7.649261959768939

In [11]:
df2= pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")

In [13]:
# deriving trip duration in minutes
df2['duration']= df2['tpep_dropoff_datetime']-df2['tpep_pickup_datetime']
df2['duration']=df2['duration'].apply(lambda x: x.total_seconds()/60)

In [14]:
#Filter dataset to contain only trips with duration between 1 and 60 minutes
new_df2=df2[(df2['duration']>= 1) & (df2['duration']<= 60)]


In [15]:
variables=new_df2[['PULocationID' ,'DOLocationID']].astype(str)
test_dict=variables.to_dict(orient='records')

In [16]:

x_test=dv.transform(test_dict)
x_test.shape

(2855951, 515)

In [17]:
y_test=new_df2['duration']
y_pred=model.predict(x_test)

#root mean square on february data
rmse=math.sqrt(mean_squared_error(y_test, y_pred))
rmse

7.811816826653192